#### Imports 

In [180]:
import random
from pathlib import Path

import joblib
import numpy as np
import optuna
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from optuna.samplers import TPESampler
from sklearn.preprocessing import RobustScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import plotly.graph_objects as go
from scipy import stats

#### ---- CONFIG ----

In [181]:
FILE_PATH = "/master_data/master_data_merged.csv"
REGION = "SE2"
target_vars = [f"Balancing_ActivatedBalancingEnergy_{REGION}_mFRR_NotSpecifiedDownActivatedVolume"]

USE_OPTUNA = True          
N_TRIALS = 50              # Number of Optuna trials INCREASE IN COLLAB
MAX_EPOCHS_OPTUNA = 10     # Epochs *per trial* INCREASE IN COLLAB
PATIENCE_OPTUNA = 5        # Early‑stopping within a trial
EPOCHS_FINAL_TRAINING = 100
PATIENCE_FINAL_TRAINING = 10
RANDOM_SEED = 69

# Flags – kept `False` for system parameters only
CREATE_INDICATOR_FEATURES = False
CREATE_CYCLICAL_FEATURES = False
CREATE_LAGGED_FEATURES = False

#### ---- DEVICE ----

In [ ]:
if torch.backends.mps.is_available():
    DEVICE = torch.device("mps")
    print("Using MPS device")
elif torch.cuda.is_available():
    DEVICE = torch.device("cuda")
    print("Using CUDA device")
else:
    DEVICE = torch.device("cpu")
    print("Using CPU device")

#### ---- PERSISTENCE -----

In [183]:
import json, joblib, platform, sys, getpass, socket, datetime

def save_best_params(best_params: dict, fname: str = "best_params.json"):
    """Augment hyper-params with reproducibility metadata and write to JSON."""
    meta = {
        "random_seed": RANDOM_SEED,
        "device": str(DEVICE),
        "torch_version": torch.__version__,
        "numpy_version": np.__version__,
        "python_version": sys.version.split()[0],
        "hostname": socket.gethostname(),
        "user": getpass.getuser(),
        "timestamp_utc": datetime.datetime.now(datetime.timezone.utc).isoformat(timespec="seconds"),
        "deterministic_cudnn": True, 
        "benchmark_cudnn": False, 
    }
    with open(fname, "w") as f:
        json.dump({**best_params, **meta}, f, indent=2)

#### ---- REPRODUCIBILITY ----

In [184]:
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark     = False

#### ---- PRE-DEFINED PLOTS ----

In [185]:
# --------------------------- NEW PLOTTING FUNCTIONS -------------------------
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf
import matplotlib.pyplot as plt
import optuna
# Functions from the provided plots.py
# ------------------------------------------------------------
def plot_loss_curve(train_history_list, val_history_list): # Renamed to avoid conflict if variables are named the same
    fig = go.Figure()
    fig.add_trace(go.Scatter(y=train_history_list, name="Train"))
    fig.add_trace(go.Scatter(y=val_history_list,   name="Val"))
    fig.update_layout(title="Training vs Validation Loss",
                      xaxis_title="Epoch", yaxis_title="Loss",
                      template="plotly_white")
    fig.show()

# ------------------------------------------------------------
def plot_optuna_history(study_obj: optuna.study.Study): # Renamed to avoid conflict
    fig = optuna.visualization.plot_optimization_history(study_obj)
    fig.update_layout(title="Optuna Optimisation History")
    fig.show()

def plot_optuna_importance(study_obj: optuna.study.Study): # Renamed to avoid conflict
    fig = optuna.visualization.plot_param_importances(study_obj)
    fig.update_layout(title="Hyper-parameter Importance (Optuna)")
    fig.show()

# ------------------------------------------------------------
def scatter_true_vs_pred(y_true, y_pred, title_suffix=""):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=y_true, y=y_pred,
                             mode="markers", opacity=0.4, name="Samples"))
    min_v, max_v = float(np.min(y_true)), float(np.max(y_true)) # Ensure y_true is not empty
    fig.add_shape(type="line", x0=min_v, y0=min_v, x1=max_v, y1=max_v,
                  line=dict(dash="dash", color="grey"))
    fig.update_layout(title=f"Scatter True vs Pred{title_suffix}",
                      xaxis_title="True", yaxis_title="Predicted",
                      template="plotly_white")
    fig.show()

# ------------------------------------------------------------
def residuals_vs_fitted(y_pred, residuals, title_suffix=""):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=y_pred, y=residuals,
                             mode="markers", opacity=0.4, name="Residuals"))
    fig.update_layout(title=f"Residuals vs Fitted{title_suffix}",
                      xaxis_title="Predicted",
                      yaxis_title="Residual (True − Pred)",
                      template="plotly_white",
                      shapes=[dict(type="line", x0=min(y_pred) if len(y_pred) > 0 else 0, 
                                   x1=max(y_pred) if len(y_pred) > 0 else 0,
                                   y0=0, y1=0, line=dict(dash="dash", color="grey"))])
    fig.show()

# ------------------------------------------------------------
def plot_acf_residuals(residuals, lags=48, title_suffix=""):
    # Uses matplotlib – quick static figure for appendix
    if len(residuals) > lags and len(residuals) > 0 : # Check if there's enough data
        plt.figure(figsize=(6,3))
        plot_acf(residuals, lags=lags)
        plt.title(f"ACF of Residuals{title_suffix}")
        plt.tight_layout()
        plt.show()
    else:
        print(f"Not enough data to plot ACF of residuals{title_suffix}. Need > {lags} data points, got {len(residuals)}.")

# ------------------------------------------------------------
def precision_recall_curve_plot(y_true_bool, y_score, title="PR curve (spike detection)"): # Renamed y_true to y_true_bool
    """Assumes y_true_bool is boolean spike indicator; y_score = |error| or model score."""
    from sklearn.metrics import precision_recall_curve, auc # Local import for clarity
    precision, recall, thresh = precision_recall_curve(y_true_bool, y_score)
    pr_auc = auc(recall, precision)
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=recall, y=precision, mode="lines",
                             name=f"PR curve (AUC={pr_auc:.3f})"))
    fig.update_layout(title=title, xaxis_title="Recall", yaxis_title="Precision",
                      template="plotly_white")
    fig.show()

# ------------------------------------------------------------
def rmse_bar(models: dict, title="RMSE Comparison"):
    """
    models = {"LSTM": rmse1, "Naive": rmse2, ...}
    """
    fig = px.bar(x=list(models.keys()), y=list(models.values()),
                 labels={"x":"Model", "y":"RMSE"},
                 title=title)
    fig.update_layout(template="plotly_white")
    fig.show()

# ------------------------------------------------------------
def radar_multi_metric(metric_dict: dict, title="Radar Comparison"):
    """
    metric_dict:
        {"LSTM": {"RMSE":0.1,"MAE":0.2,"R2":0.9},
         "Naive":{...}, ...}
    """
    metrics = list(next(iter(metric_dict.values())).keys()) if metric_dict else []
    if not metrics:
        print("Metric dictionary is empty or malformed for radar plot.")
        return
    fig = go.Figure()
    for name, vals in metric_dict.items():
        fig.add_trace(go.Scatterpolar(
            r=[vals.get(m, np.nan) for m in metrics], # Use .get for safety
            theta=metrics,
            fill="toself",
            name=name))
    fig.update_layout(title=title,
                      polar=dict(radialaxis=dict(visible=True)),
                      template="plotly_white",
                      showlegend=True)
    fig.show()


#### ---- DATA LOADING ----

In [ ]:
print("Loading data …")
df_all = pd.read_csv(FILE_PATH)

price_column_name_to_plot = f"Balancing_PricesOfActivatedBalancingEnergy_{REGION}_mFRR_NotSpecifiedDownPrice"

# Store price data separately before modifying df_all or creating df
price_data_full = None
if price_column_name_to_plot in df_all.columns:
    price_data_full = df_all[['DateTime', price_column_name_to_plot]].copy()
    price_data_full['DateTime'] = pd.to_datetime(price_data_full['DateTime'], utc=True)
    price_data_full.set_index('DateTime', inplace=True)
    print(f"Stored '{price_column_name_to_plot}' separately for plotting.")
else:
    print(f"Warning: Price column '{price_column_name_to_plot}' not found in df_all. Will not be plotted.")

# Bidding zone filter
from filter_features import pick_region_filter
BDZ_filter = pick_region_filter(region=REGION, remove_balancing=True)

drop_list = [
    f"Balancing_PricesOfActivatedBalancingEnergy_{REGION}_mFRR_NotSpecifiedUpPrice",
    f"Balancing_PricesOfActivatedBalancingEnergy_{REGION}_mFRR_NotSpecifiedDownPrice",
    f"Balancing_ActivatedBalancingEnergy_{REGION}_mFRR_NotSpecifiedUpActivatedVolume",
    f"Balancing_ActivatedBalancingEnergy_{REGION}_mFRR_NotSpecifiedDownActivatedVolume",
]
BDZ_filter = [c for c in BDZ_filter if c not in drop_list or c in target_vars]

df = df_all[["DateTime"] + BDZ_filter].copy()
df["DateTime"] = pd.to_datetime(df["DateTime"], utc=True)
df.set_index("DateTime", inplace=True)
df = df.asfreq("h")


# Feature Engineering Logic
new_indicator_features = []
if CREATE_INDICATOR_FEATURES:
    # Feature Engineering: Add indicator for missing values (before other feature engineering)
    # These indicators capture missingness introduced by asfreq or originally present
    indicator_features_to_create = [col for col in BDZ_filter + target_vars if col in df.columns]
    for col in indicator_features_to_create:
        df[f'{col}_was_missing'] = df[col].isnull().astype(int)
    new_indicator_features = [f'{col}_was_missing' for col in indicator_features_to_create]

new_time_features = []
if CREATE_CYCLICAL_FEATURES:
    # Feature Engineering: Add cyclical time features
    df['hour_sin'] = np.sin(2 * np.pi * df.index.hour / 24.0)
    df['hour_cos'] = np.cos(2 * np.pi * df.index.hour / 24.0)
    df['dayofweek_sin'] = np.sin(2 * np.pi * df.index.dayofweek / 7.0)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df.index.dayofweek / 7.0)
    df['weekofyear_sin'] = np.sin(2 * np.pi * df.index.isocalendar().week / 52.0) # Using isocalendar().week
    df['weekofyear_cos'] = np.cos(2 * np.pi * df.index.isocalendar().week / 52.0)
    new_time_features = ['hour_sin', 'hour_cos', 'dayofweek_sin', 'dayofweek_cos', 'weekofyear_sin', 'weekofyear_cos']

new_lagged_features = []
if CREATE_LAGGED_FEATURES:
    # Feature Engineering: Add lagged target features
    # Ensure target_vars is defined and contains the column names to lag
    for target_var in target_vars: # Iterate if multiple targets, though current setup has one
        df[f'{target_var}_lag1'] = df[target_var].shift(1)
        df[f'{target_var}_lag24'] = df[target_var].shift(24)
        df[f'{target_var}_lag168'] = df[target_var].shift(168)
    for target_var in target_vars:
        new_lagged_features.extend([f'{target_var}_lag1', f'{target_var}_lag24', f'{target_var}_lag168'])

df.dropna(axis=1, how="all", inplace=True)  # Drop columns with all NaNs

exogenous_vars = [c for c in BDZ_filter if c not in target_vars]

# Build the full list of exogenous variables based on flags
if CREATE_CYCLICAL_FEATURES:
    exogenous_vars.extend(new_time_features)
if CREATE_LAGGED_FEATURES:
    exogenous_vars.extend(new_lagged_features)
if CREATE_INDICATOR_FEATURES:
    exogenous_vars.extend(new_indicator_features)

# Ensure all exogenous_vars are present in df.columns and are unique
exogenous_vars = sorted(list(set(var for var in exogenous_vars if var in df.columns)))

#### ---- SPLIT BY DATE ----

In [ ]:
df_train = df.loc[:"2023-12-31"].copy()
df_val   = df.loc["2024-01-01":"2025-01-10"].copy()
df_val2  = df.loc["2024-01-11":"2025-01-31"].copy() # New validation set for final training early stopping
df_test  = df.loc["2025-02-01":].copy()

print(
    f"Train {df_train.shape}, Val {df_val.shape}, Val2 {df_val2.shape}, Test {df_test.shape}")

#### ---- SCALERS ----

In [ ]:
scaler_X = RobustScaler(quantile_range=(5.0, 95.0))
scaler_y = RobustScaler(quantile_range=(5.0, 95.0))
scaler_X.fit(df_train[exogenous_vars])
scaler_y.fit(df_train[target_vars])

# Helper to scale a split -----------------------------------------------------

def scale_split(split_df: pd.DataFrame) -> pd.DataFrame:
    X_scaled = scaler_X.transform(split_df[exogenous_vars])
    y_scaled = scaler_y.transform(split_df[target_vars])
    return pd.DataFrame(
        np.hstack([X_scaled, y_scaled]),
        index=split_df.index,
        columns=exogenous_vars + target_vars,
    )

#### ---- SEQUENCE BUILDER ----

In [189]:
def create_sequences_torch(data_df: pd.DataFrame, target_idx, feature_idx, lookback: int, horizon: int = 1):
    data_tensor = torch.tensor(data_df.values, dtype=torch.float32)
    num_samples = data_tensor.shape[0] - lookback - horizon
    X = torch.zeros((num_samples, lookback, len(feature_idx)), dtype=torch.float32)
    y = torch.zeros((num_samples, horizon, len(target_idx)), dtype=torch.float32)
    for i in range(num_samples):
        X[i] = data_tensor[i : i + lookback, feature_idx]
        y[i] = data_tensor[i + lookback : i + lookback + horizon, target_idx]
    return X, y

#### ---- MODEL & LOSS ----

In [190]:

class MaskedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=1, dense_size=64, output_size=1, horizon=1, dropout=0.2, dropout_lstm=0.2, bidirectional=False):
        super().__init__()
        self.horizon = horizon
        self.output_size = output_size
        actual_in = input_size * 2
        self.lstm = nn.LSTM(
            input_size=actual_in,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout_lstm if num_layers > 1 else 0.0,
            bidirectional=bidirectional,
            batch_first=True,
        )
        hidden_out = hidden_size * (2 if bidirectional else 1)
        self.fc1 = nn.Linear(hidden_out, dense_size)
        self.fc2 = nn.Linear(dense_size, horizon * output_size)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(dropout)
        self.drop_lstm = nn.Dropout(dropout_lstm)

    def forward(self, x, mask):
        x = torch.cat((x, mask), dim=2)
        out, _ = self.lstm(x)
        out = self.drop_lstm(out)
        out = self.relu(self.fc1(out[:, -1, :]))
        out = self.drop(out)
        out = self.fc2(out)
        return out.view(out.size(0), self.horizon, self.output_size)


# def masked_mse_loss(y_pred, y_true, mask, alpha: float = 1.0):
#     se = (y_pred - y_true) ** 2
#     weights = 1.0 + alpha * torch.abs(y_true.detach())
#     loss = se * weights * mask
#     valid = mask.sum()
#     return loss.sum() / valid if valid > 0 else torch.tensor(0.0, device=y_pred.device, requires_grad=True)


# def masked_mse_loss(y_pred, y_true_scaled, mask, alpha=1.0):
#     scale  = torch.tensor(scaler_y.scale_,  device=y_pred.device,
#                           dtype=y_pred.dtype).view(1, 1, -1)
#     center = torch.tensor(scaler_y.center_, device=y_pred.device,
#                           dtype=y_pred.dtype).view(1, 1, -1)

#     y_true_unscaled = y_true_scaled * scale + center          # (B,H,D)
#     weights = 1.0 + alpha * torch.abs(y_true_unscaled).detach()

#     se   = (y_pred - y_true_scaled) ** 2
#     loss = (se * weights * mask).sum() / mask.sum()
#     return loss

scale_t  = torch.tensor(scaler_y.scale_[0],  device=DEVICE, dtype=torch.float32)
center_t = torch.tensor(scaler_y.center_[0], device=DEVICE, dtype=torch.float32)

def weighted_mse_loss(y_pred, y_true_scaled, mask, alpha=10.0):
    """
    y shapes  (B, 1)  because you still have one target
    mask same shape, 1 for valid, 0 for padded
    """
    y_true_unscaled = y_true_scaled * scale_t + center_t
    weights = 1.0 + alpha * torch.abs(y_true_unscaled)
    se      = (y_pred - y_true_scaled) ** 2
    loss    = (se * weights * mask).sum() / mask.sum()
    return loss


#### ---- OPTUNA OBJECTIVE ----

In [191]:

def optuna_objective(trial: optuna.Trial) -> float:
    # Sample hyper‑params
    params = {
        "hidden_size": trial.suggest_categorical("hidden_size", [32, 64, 128, 256]),
        "num_layers": trial.suggest_int("num_layers", 1, 3),
        "bidirectional": trial.suggest_categorical("bidirectional", [False, True]),
        "dense_size": trial.suggest_categorical("dense_size", [32, 64, 128]),
        "dropout": trial.suggest_float("dropout", 0.0, 0.5),
        "dropout_lstm": trial.suggest_float("dropout_lstm", 0.0, 0.5),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 5e-3, log=True),
        # "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [64, 128, 256, 512]),
        "seq_length": trial.suggest_categorical("seq_length", choices=[24, 72, 168]),
    }

    # Build scaled splits each trial to allow varying seq_length
    train_scaled = scale_split(df_train)
    val_scaled   = scale_split(df_val)

    target_idx  = [train_scaled.columns.get_loc(c) for c in target_vars]
    feature_idx = [train_scaled.columns.get_loc(c) for c in exogenous_vars]

    X_train, y_train = create_sequences_torch(train_scaled, target_idx, feature_idx, params["seq_length"], 1)
    X_val, y_val     = create_sequences_torch(val_scaled,   target_idx, feature_idx, params["seq_length"], 1)

    # Masks & NaN replacement (there shouldn't be NaNs after fillna, but keep generic)
    X_mask_train = (~torch.isnan(X_train)).float(); X_train = torch.nan_to_num(X_train)
    y_mask_train = (~torch.isnan(y_train)).float(); y_train = torch.nan_to_num(y_train)
    X_mask_val   = (~torch.isnan(X_val)).float();   X_val   = torch.nan_to_num(X_val)
    y_mask_val   = (~torch.isnan(y_val)).float();   y_val   = torch.nan_to_num(y_val)

    train_loader = DataLoader(TensorDataset(X_train, X_mask_train, y_train, y_mask_train), batch_size=params["batch_size"], shuffle=False)
    val_loader   = DataLoader(TensorDataset(X_val,   X_mask_val,   y_val,   y_mask_val),   batch_size=params["batch_size"], shuffle=False)

    model = MaskedLSTM(
        input_size=len(feature_idx),
        hidden_size=params["hidden_size"],
        num_layers=params["num_layers"],
        dense_size=params["dense_size"],
        output_size=len(target_vars),
        dropout=params["dropout"],
        dropout_lstm=params["dropout_lstm"],
        bidirectional=params["bidirectional"],
    ).to(DEVICE)

    optimiser = optim.Adam(model.parameters(), lr=params["learning_rate"])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimiser, mode="min", patience=2, factor=0.5)

    best_val = float("inf")
    patience = 0

    for epoch in range(MAX_EPOCHS_OPTUNA):
        # ---- train ----
        model.train(); train_loss = 0.0
        for Xb, Xm, yb, ym in train_loader:
            Xb, Xm, yb, ym = Xb.to(DEVICE), Xm.to(DEVICE), yb.to(DEVICE), ym.to(DEVICE)
            optimiser.zero_grad()
            y_pred = model(Xb, Xm)
            
            # loss = masked_mse_loss(y_pred, yb, ym)
            loss = weighted_mse_loss(y_pred, yb, ym)
            
            loss.backward(); torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimiser.step(); train_loss += loss.item()
        train_loss /= len(train_loader)

        # ---- val ----
        model.eval(); val_loss = 0.0
        with torch.no_grad():
            for Xb, Xm, yb, ym in val_loader:
                Xb, Xm, yb, ym = Xb.to(DEVICE), Xm.to(DEVICE), yb.to(DEVICE), ym.to(DEVICE)
                y_pred = model(Xb, Xm)

                # val_loss += masked_mse_loss(y_pred, yb, ym).item()
                val_loss += weighted_mse_loss(y_pred, yb, ym).item()

        val_loss /= len(val_loader)
        scheduler.step(val_loss)

        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        if val_loss < best_val:
            best_val = val_loss; patience = 0
        else:
            patience += 1
            if patience >= PATIENCE_OPTUNA:
                break

    return best_val

#### ---- HYPER‑PARAM SELECTION  ----

In [ ]:
if USE_OPTUNA:
    print("\n=== Running Optuna study ===")
    sampler = TPESampler(seed=RANDOM_SEED)
    study = optuna.create_study(direction="minimize", sampler=sampler)
    study.optimize(optuna_objective, n_trials=N_TRIALS, show_progress_bar=True)
    best_params = study.best_trial.params
    print("Best params:")
    for k, v in best_params.items():
        print(f"  {k}: {v}")

    save_best_params(best_params)
    joblib.dump(study, "study.pkl")

    # Plot Optuna optimization history
    fig_opt_hist = optuna.visualization.plot_optimization_history(study)
    fig_opt_hist.show()

    # The existing call optuna.visualization.plot_optimization_history(study).show() is fine.
    # The new plot_optuna_history(study) can be used if preferred or if the layout modification is desired.
    # For now, let's add the new hyper-parameter importance plot call:
    plot_optuna_importance(study) # Call the new function for hyper-parameter importance
else:
    # Manual defaults from your notebook
    best_params = {
        "hidden_size": 256,
        "num_layers": 2,
        "bidirectional": True,
        "dense_size": 128,
        "dropout": 0.33194731105235564,
        "dropout_lstm": 0.3447348446997174,
        "learning_rate": 0.0006857135933869154,
        "batch_size": 256,
        "seq_length": 36,
    }
    save_best_params(best_params) # Save even if not using Optuna, for consistency

#### ---- DATA REBUILD WITH BEST PARAMS ----

In [ ]:
print("\nRe‑building datasets with best hyper‑parameters …")
# Train on train + original val, validate final model on val2
train_val_combined_df = pd.concat([df_train, df_val])
train_val_scaled = scale_split(train_val_combined_df) # Scale the combined train+val
val2_scaled = scale_split(df_val2) # Scale val2

target_idx  = [train_val_scaled.columns.get_loc(c) for c in target_vars]
feature_idx = [train_val_scaled.columns.get_loc(c) for c in exogenous_vars]

X_tv, y_tv = create_sequences_torch(train_val_scaled, target_idx, feature_idx, best_params["seq_length"], 1)
X_val2_tensor, y_val2_tensor = create_sequences_torch(val2_scaled, target_idx, feature_idx, best_params["seq_length"], 1) # Sequences for val2
X_test_scaled = scale_split(df_test)
X_test_tensor, y_test_tensor = create_sequences_torch(X_test_scaled, target_idx, feature_idx, best_params["seq_length"], 1)

# Masks & NaNs
X_mask_tv     = (~torch.isnan(X_tv)).float();     X_tv     = torch.nan_to_num(X_tv)
y_mask_tv     = (~torch.isnan(y_tv)).float();     y_tv     = torch.nan_to_num(y_tv)
X_mask_val2   = (~torch.isnan(X_val2_tensor)).float(); X_val2_tensor = torch.nan_to_num(X_val2_tensor) # Mask for val2
y_mask_val2   = (~torch.isnan(y_val2_tensor)).float(); y_val2_tensor = torch.nan_to_num(y_val2_tensor) # Mask for val2
X_mask_test   = (~torch.isnan(X_test_tensor)).float(); X_test_tensor = torch.nan_to_num(X_test_tensor)
y_mask_test   = (~torch.isnan(y_test_tensor)).float(); y_test_tensor = torch.nan_to_num(y_test_tensor)

train_loader = DataLoader(TensorDataset(X_tv, X_mask_tv, y_tv, y_mask_tv),
                          batch_size=best_params["batch_size"], shuffle=False)
val2_loader = DataLoader(TensorDataset(X_val2_tensor, X_mask_val2, y_val2_tensor, y_mask_val2), # Loader for val2
                         batch_size=best_params["batch_size"], shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor, X_mask_test, y_test_tensor, y_mask_test),
                         batch_size=best_params["batch_size"], shuffle=False)



#### ---- FINAL MODEL TRAIN ----

In [ ]:

print("\nTraining final model …")
model = MaskedLSTM(
    input_size=len(feature_idx),
    hidden_size=best_params["hidden_size"],
    num_layers=best_params["num_layers"],
    dense_size=best_params["dense_size"],
    output_size=len(target_vars),
    dropout=best_params["dropout"],
    dropout_lstm=best_params["dropout_lstm"],
    bidirectional=best_params["bidirectional"],
).to(DEVICE)

optimizer = optim.Adam(model.parameters(), lr=best_params["learning_rate"])
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

# Training parameters
epochs = EPOCHS_FINAL_TRAINING
patience_early_stop = PATIENCE_FINAL_TRAINING
best_val_loss = float('inf')
epochs_no_improve = 0
best_model_path = 'best_model.pth'
scaler_X_path = 'scaler_X.joblib'
scaler_y_path = 'scaler_y.joblib'
loss_alpha = 1.0  # Alpha for weighted MSE

train_history, val_history = [], []        

for epoch in range(epochs):
    model.train()
    train_loss = 0
    for X_batch, X_mask_batch, y_batch, y_mask_batch in train_loader:
        X_batch, X_mask_batch, y_batch, y_mask_batch = (
            X_batch.to(DEVICE),
            X_mask_batch.to(DEVICE),
            y_batch.to(DEVICE),
            y_mask_batch.to(DEVICE),
        )
        optimizer.zero_grad()
        y_pred = model(X_batch, X_mask_batch)

        # loss = masked_mse_loss(y_pred, y_batch, y_mask_batch, alpha=loss_alpha)
        loss = weighted_mse_loss(y_pred, y_batch, y_mask_batch)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    # Validation - use val2_loader for early stopping and scheduler
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for X_batch, X_mask_batch, y_batch, y_mask_batch in val2_loader: # Use val2_loader
            X_batch, X_mask_batch, y_batch, y_mask_batch = (
                X_batch.to(DEVICE),
                X_mask_batch.to(DEVICE),
                y_batch.to(DEVICE),
                y_mask_batch.to(DEVICE),
            )
            y_pred = model(X_batch, X_mask_batch)
            
            # val_loss += masked_mse_loss(y_pred, y_batch, y_mask_batch, alpha=loss_alpha).item()
            val_loss += weighted_mse_loss(y_pred, y_batch, y_mask_batch).item()


    avg_val_loss = val_loss / len(val2_loader)

    # Print the losses for this epoch
    print(f"Epoch {epoch+1:02d}: Train Loss = {avg_train_loss:.10f}, Val Loss = {avg_val_loss:.10f}")

    train_history.append(avg_train_loss)     # 
    val_history.append(avg_val_loss)         # 

    # Step the scheduler
    scheduler.step(avg_val_loss)

    # ----- Early Stopping Check -----
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        joblib.dump(scaler_X, scaler_X_path)
        joblib.dump(scaler_y, scaler_y_path)
        epochs_no_improve = 0
        print(f"New best model saved with validation loss: {best_val_loss:.10f}")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience_early_stop:
            print(f"Early stopping triggered. No improvement in validation loss for {patience_early_stop} consecutive epochs.")
            break

# Load the best model
model.load_state_dict(torch.load(best_model_path))

# Persist the curves – for convergence plots
pd.DataFrame({"train_loss": train_history,
              "val_loss":   val_history}).to_csv("loss_history.csv", index_label="epoch")

# Call the new loss curve plot function
plot_loss_curve(train_history, val_history)

#### ---- PREDICTION & EVALUATION ----

In [ ]:
model.eval()

# Generate predictions for test set
all_preds_list = []

with torch.no_grad():
    for X_batch, X_mask_batch, _, _ in test_loader:
        # Move data to the same device as the model
        X_batch = X_batch.to(DEVICE)
        X_mask_batch = X_mask_batch.to(DEVICE)
        
        # Generate predictions
        y_pred = model(X_batch, X_mask_batch)
        
        # Append predictions (move to CPU and convert to numpy)
        all_preds_list.append(y_pred.cpu().numpy())

# Concatenate batches along the first axis (samples)
all_preds_array = np.concatenate(all_preds_list, axis=0)  # Shape: (num_samples, horizon, num_model_outputs)

# For plotting, let's select the first time step prediction for each sequence
model_predictions_raw = all_preds_array[:, 0, :]  # Shape: (num_samples, num_model_outputs)

# Invert the scaling on predictions if a scaler was used for y
model_predictions_unscaled = scaler_y.inverse_transform(model_predictions_raw)

# Get the datetime indices for the predictions
# The test index starts at seq_length timesteps after the start of the test set
test_dates = df_test.index[best_params["seq_length"]:best_params["seq_length"]+len(all_preds_array)]

# Get the actual values directly from the original dataframe for the TARGET_VARS
true_values_from_df_test = df_test.loc[test_dates, target_vars].values

# Ensure true_series_scaled is 2D, especially if target_vars has only one element
if true_values_from_df_test.ndim == 1:
    true_values_from_df_test = true_values_from_df_test.reshape(-1, 1)

# Now, inverse transform the SCALED true values to get UN SCALED true values
# CORRECTED: true_values_from_df_test already contains unscaled values from df_test.
# No inverse_transform needed here.
true_series_unscaled = true_values_from_df_test

# Ensure no nans in the final unscaled true values for plotting/metrics
true_series_unscaled = np.nan_to_num(true_series_unscaled, nan=0.0)

# These are the names corresponding to ALL of the model's output dimensions, in their output order.
model_output_all_target_names = target_vars

# Dump predictions vs. ground truth
pred_df  = pd.DataFrame(model_predictions_unscaled,
                        index=test_dates,
                        columns=[f"pred_{c}" for c in target_vars])
truth_df = pd.DataFrame(true_series_unscaled,
                        index=test_dates,
                        columns=[f"true_{c}" for c in target_vars])

pd.concat([pred_df, truth_df], axis=1).to_parquet("predictions.parquet")
print("Saved test-set predictions → predictions.parquet")

# --------------------------- PLOTTING & METRICS ----------------------------
print("\nEvaluating and plotting results...")

# Now, plot the predictions vs. true values for each target variable specified in `target_vars`
print(f"Evaluating targets: {target_vars}")

for i, current_target_name_to_eval in enumerate(target_vars):
    fig = go.Figure()
    
    # True values: column i from true_series_unscaled corresponds to target_vars[i]
    current_true_values = true_series_unscaled[:, i]
    
    # Predicted values: find the column in model_predictions_unscaled that corresponds to current_target_name_to_eval
    try:
        # Find the index of the current evaluation target in the list of all model output names
        model_output_idx = model_output_all_target_names.index(current_target_name_to_eval)
        current_predicted_values = model_predictions_unscaled[:, model_output_idx]
    except ValueError:
        print(f"Error: The target '{current_target_name_to_eval}' (from your `target_vars`) "
              f"was not found in `model_output_all_target_names`: {model_output_all_target_names}. "
              "Cannot plot or calculate metrics for this target. Please check your variable names.")
        continue # Skip to the next target in target_vars

    fig.add_trace(go.Scatter(
        x=test_dates,
        y=current_true_values,
        mode='lines+markers',
        name="True",
        marker=dict(symbol='circle', size=6)
    ))
    fig.add_trace(go.Scatter(
        x=test_dates,
        y=current_predicted_values,
        mode='lines+markers',
        name="Predicted",
        marker=dict(symbol='x', size=6)
    ))
    fig.update_layout(
        title=f"January 2025: True vs Predicted for {current_target_name_to_eval}",
        xaxis_title="Date",
        yaxis_title="Volume (MW)",
        template="plotly_white",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )
    fig.update_xaxes(tickformat="%d %b %Y %H:%M")
    fig.show()

# Create a figure showing all evaluated targets (from target_vars) on the same timeline
fig_combined = go.Figure()
plotted_any_combined = False

for i, current_target_name_to_eval in enumerate(target_vars):
    current_true_values = true_series_unscaled[:, i]
    try:
        model_output_idx = model_output_all_target_names.index(current_target_name_to_eval)
        current_predicted_values = model_predictions_unscaled[:, model_output_idx]
    except ValueError:
        # Error already printed in the loop above
        continue

    fig_combined.add_trace(go.Scatter(
        x=test_dates,
        y=current_true_values,
        mode='lines',
        name=f"{current_target_name_to_eval} (True)",
        line=dict(width=2)
    ))
    fig_combined.add_trace(go.Scatter(
        x=test_dates,
        y=current_predicted_values,
        mode='lines',
        line=dict(dash='dash', width=2),
        name=f"{current_target_name_to_eval} (Predicted)"
    ))
    plotted_any_combined = True

# Add price data trace if available
price_column_name = price_column_name_to_plot
price_trace_added = False
plot_name_suffix = " (Original)"

# Access the pre-stored price data using the test_dates index
if price_data_full is not None and price_column_name in price_data_full.columns:
    try:
        # Align price data with test_dates using the index
        price_data_to_plot = price_data_full.loc[test_dates, price_column_name].values.astype(float)

        # Check if alignment resulted in all NaNs (can happen with index gaps)
        if np.isnan(price_data_to_plot).all():
             print(f"Warning: Price data for test dates resulted in all NaNs after alignment.")
             price_data_to_plot = None # Prevent plotting if all NaNs

    except KeyError:
        # This might happen if test_dates contains dates not present in the original price_data_full index
        print(f"Warning: Could not align price data with test_dates. Index mismatch between price_data_full and test_dates.")
        # Attempt reindexing as a fallback, dropping dates not found in price_data_full
        try:
            aligned_prices = price_data_full.reindex(test_dates)[price_column_name]
            if not aligned_prices.isnull().all():
                price_data_to_plot = aligned_prices.values.astype(float)
                print("Successfully aligned price data using reindex.")
            else:
                print("Reindexing price data resulted in all NaNs.")
                price_data_to_plot = None
        except Exception as reindex_e:
            print(f"Error during fallback reindexing of price data: {reindex_e}")
            price_data_to_plot = None
    except Exception as e:
        print(f"An error occurred accessing or aligning stored price data: {e}")
        price_data_to_plot = None
else:
     # This case is hit if price_data_full was None initially
     # The warning was already printed when price_data_full was defined
     price_data_to_plot = None # Ensure it's None

if price_data_to_plot is not None:
    try:
        fig_combined.add_trace(go.Scatter(
            x=test_dates,
            y=price_data_to_plot,
            mode='lines',
            name=f"mFRR Down Price ({REGION}){plot_name_suffix}",
            yaxis="y2",
            line=dict(width=1.5, color='rgba(255,165,0,0.7)') # Orange color for price
        ))
        price_trace_added = True
        plotted_any_combined = True # Ensure combined plot is shown if price trace is added
    except Exception as e:
        print(f"An error occurred attempting to add price trace to plot: {e}")
        price_trace_added = False # Ensure flag is false if adding trace fails

# If plotting failed or data was None, print a message
if not price_trace_added:
    print(f"Info: Price data ('{price_column_name}') could not be prepared or added to the plot.")

# ADDITION END

if plotted_any_combined:
    layout_args = dict(
        title=f"January 2025 ({REGION}): True vs Predicted Targets, with Price Data",
        xaxis_title="Date",
        yaxis=dict(title="Volume (MW)"), # This is yaxis (y1)
        template="plotly_white",
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02, # Default y for legend
            xanchor="right",
            x=1
        )
    )
    if price_trace_added:
        layout_args['yaxis2'] = dict(
            title="Price (€/MWh)", # Adjust unit if known, placeholder
            overlaying='y',
            side='right',
            showgrid=False, # Optional: hide grid for secondary axis
            rangemode='tozero' # Assumes prices are generally positive or start from zero
        )
        # Adjust legend position slightly if yaxis2 is present, to avoid overlap with its title
        layout_args['legend']['y'] = 1.08 
    
    fig_combined.update_layout(**layout_args)
    fig_combined.update_xaxes(tickformat="%d %b %Y %H:%M")
    fig_combined.show()
else:
    print("No targets were plotted in the combined figure due to previous errors or empty `target_vars`.")


# Calculate performance metrics
for i, current_target_name_to_eval in enumerate(target_vars):
    current_true_values = true_series_unscaled[:, i]
    try:
        model_output_idx = model_output_all_target_names.index(current_target_name_to_eval)
        current_predicted_values = model_predictions_unscaled[:, model_output_idx]
    except ValueError:
        # Error already printed, skip metrics for this target
        continue
        
    mse = mean_squared_error(current_true_values, current_predicted_values)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(current_true_values, current_predicted_values)
    r2 = r2_score(current_true_values, current_predicted_values)
    
    print(f"\nPerformance Metrics for {current_target_name_to_eval}:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R²: {r2:.4f}")

    # Spike Metrics Calculation
    # 1. Determine spike threshold from unscaled training data for the current target
    # We need to access the original df_train for this target *before* it was scaled.
    # This requires us to re-establish df_train or have saved its original values.
    # For simplicity, let's use the P90 of the absolute values of the *current test set's true values* as a proxy for now.
    # A more robust approach would be to calculate this on df_train and pass it down or re-calculate properly.
    # However, to avoid complex data refetching at this stage, we use current_true_values for threshold calc.
    if len(current_true_values) > 0:
        spike_threshold = np.percentile(np.abs(current_true_values), 90) 
        print(f"Spike Threshold (P90 of test set abs values) for {current_target_name_to_eval}: {spike_threshold:.4f}")

        true_spikes = np.abs(current_true_values) > spike_threshold
        pred_spikes = np.abs(current_predicted_values) > spike_threshold

        tp = np.sum(true_spikes & pred_spikes)
        fp = np.sum(~true_spikes & pred_spikes)
        fn = np.sum(true_spikes & ~pred_spikes)

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        print(f"Spike Metrics for {current_target_name_to_eval} (Threshold > {spike_threshold:.2f}):")
        print(f"  True Positives (Spikes correctly predicted): {tp}")
        print(f"  False Positives (Non-spikes predicted as spikes): {fp}")
        print(f"  False Negatives (Spikes missed): {fn}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall: {recall:.4f}")
        print(f"  F1-Score: {f1_score:.4f}")
    else:
        print(f"Not enough data to calculate spike metrics for {current_target_name_to_eval}.")

# Additional analysis to check for potential data leakage indicators
all_errors_data = {} # To store errors for potential later use (e.g. constructing error_df)

for i, current_target_name_to_eval in enumerate(target_vars):
    current_true_values = true_series_unscaled[:, i]
    try:
        model_output_idx = model_output_all_target_names.index(current_target_name_to_eval)
        current_predicted_values = model_predictions_unscaled[:, model_output_idx]
    except ValueError:
        continue # Skip if target name not found in model outputs
    
    errors = current_true_values - current_predicted_values
    all_errors_data[f'error_{current_target_name_to_eval}'] = errors # Store errors
    
    # Call new plotting functions for current target
    scatter_true_vs_pred(current_true_values, current_predicted_values, title_suffix=f" for {current_target_name_to_eval}")
    residuals_vs_fitted(current_predicted_values, errors, title_suffix=f" for {current_target_name_to_eval}")
    plot_acf_residuals(errors, title_suffix=f" for {current_target_name_to_eval}")
    
    # Plot error distribution
    fig_err_dist = go.Figure()
    fig_err_dist.add_trace(go.Histogram(x=errors, nbinsx=50))
    fig_err_dist.update_layout(
        title=f"Error Distribution for {current_target_name_to_eval}",
        xaxis_title="Error (True - Predicted)",
        yaxis_title="Frequency",
        template="plotly_white"
    )
    fig_err_dist.show()
    
    # Test for normal distribution of errors
    if len(errors) >= 8: # normaltest requires at least 8 samples
        k2, p_norm_test = stats.normaltest(errors)
        print(f"\nNormality test for {current_target_name_to_eval} errors:")
        print(f"K²: {k2:.4f}, p-value: {p_norm_test:.4f}")
        if p_norm_test < 0.05:
            print("Error distribution is not normal - may indicate issues with model")
        else:
            print("Error distribution appears normal")
    else:
        print(f"\nNormality test for {current_target_name_to_eval} errors: Not enough samples (need >= 8, got {len(errors)})")
        
    # Plot error time series
    fig_err_ts = go.Figure()
    fig_err_ts.add_trace(go.Scatter(
        x=test_dates,
        y=errors,
        mode='lines',
        name="Prediction Error"
    ))
    fig_err_ts.update_layout(
        title=f"Prediction Errors Over Time for {current_target_name_to_eval}",
        xaxis_title="Date",
        yaxis_title="Error (True - Predicted)",
        template="plotly_white"
    )
    fig_err_ts.show()

# If you need an error DataFrame for further analysis (e.g., autocorrelation with statsmodels)
if all_errors_data:
    error_df_evaluated_targets = pd.DataFrame(all_errors_data, index=test_dates)
else:
    print("\nNo errors were collected for DataFrame construction (all_errors_data is empty).")


